In [1]:
import pandas as pd
import numpy as np

In [52]:
df = pd.read_parquet('tiling_results.parquet')
df.shape

(100, 14)

In [53]:
len(df[df.tiling_compatible == True]) 

58

In [50]:
counts = df['incompatible_reason'].value_counts(dropna=False).reset_index()
counts.columns = ['Value', 'Count']
print(counts)

                    Value  Count
0                    None     58
1          cant_open_file     13
2  cant_extract_variables     13
3  tile_generation_failed     11
4                 timeout      3
5      unsupported_format      2


In [45]:
df[df.incompatible_reason == "tile_generation_failed"]

,collection_concept_id,concept_id,data_center,data_url,backend,format,extension,tiling_compatible,incompatible_reason,error_message,tiles_url,variable,data_variables,num_granules
15,C1239900200-GES_DISC,G1266287322-GES_DISC,NASA/GSFC/SED/ESD/TISL/GESDISC,s3://gesdisc-cumulus-prod-protected/DeepBlueSe...,xarray,HDF5,h5,False,tile_generation_failed,No group found in store <fsspec.mapping.FSMap ...,https://staging.openveda.cloud/api/titiler-cmr...,aerosol_optical_thickness_550_count_land,"[aerosol_optical_thickness_550_count_land, aer...",4520.0
26,C1239900190-GES_DISC,G1266288592-GES_DISC,NASA/GSFC/SED/ESD/TISL/GESDISC,s3://gesdisc-cumulus-prod-protected/DeepBlueSe...,xarray,HDF5,h5,False,tile_generation_failed,No group found in store <fsspec.mapping.FSMap ...,https://staging.openveda.cloud/api/titiler-cmr...,aerosol_optical_thickness_550_count_land,"[aerosol_optical_thickness_550_count_land, aer...",4520.0
29,C2215175232-GES_DISC,G2221923072-GES_DISC,NASA/GSFC/SED/ESD/TISL/GESDISC,s3://gesdisc-cumulus-prod-protected/MINDS/OMI_...,xarray,netCDF,nc,False,tile_generation_failed,Couldn't find X/Y dimensions in (),https://staging.openveda.cloud/api/titiler-cmr...,crs,"[LatitudeBounds, LongitudeBounds, TimeBounds, ...",7345.0
39,C1563089663-GES_DISC,G1563424074-GES_DISC,NASA/GSFC/SED/ESD/TISL/GESDISC,s3://gesdisc-cumulus-prod-protected/FLDAS/FLDA...,xarray,netCDF,nc,False,tile_generation_failed,"Couldn't find X/Y dimensions in ('time', 'Y', ...",https://staging.openveda.cloud/api/titiler-cmr...,Evap_tavg,"[time_bnds, Evap_tavg, LWdown_f_tavg, Lwnet_ta...",525.0
43,C1374187813-GES_DISC,G1374348686-GES_DISC,NASA/GSFC/SED/ESD/TISL/GESDISC,s3://gesdisc-cumulus-prod-protected/LANDMET/LA...,xarray,netCDF,nc,False,tile_generation_failed,"Couldn't find X/Y dimensions in ('eqcell',)",https://staging.openveda.cloud/api/titiler-cmr...,eqlon,"[eqlon, eqlat, eqlon_index, eqlat_index, eqcel...",3652.0


In [46]:
# Filter out None/NaN and empty arrays
subset_df = df[df['data_variables'].apply(
    lambda x: x is not None and (isinstance(x, np.ndarray) and (x.size > 0 and x[0] != None))
)]
subset_df.data_variables

1     [time_bnds, Swnet_tavg, Lwnet_tavg, Qle_tavg, ...
2     [time_bnds, Swnet_tavg, Lwnet_tavg, Qle_tavg, ...
6     [BSTAR, CDH, CDM, CDQ, CN, DISPH, EFLUX, EVAP,...
7     [DISPH, PS, QV10M, QV2M, SLP, T10M, T2M, TO3, ...
9     [ALBEDO, ALBNIRDF, ALBNIRDR, ALBVISDF, ALBVISD...
10    [petpen, petpt, petref, rnet, ghflx, rflag, gf...
11    [precipitation, precipitation_cnt, precipitati...
12    [EPV, H, O3, OMEGA, PHIS, PS, QI, QL, QV, RH, ...
14    [time_bnds, Swnet_tavg, Lwnet_tavg, Qle_tavg, ...
15    [aerosol_optical_thickness_550_count_land, aer...
17    [precipitation, precipitation_cnt, precipitati...
19    [AIRDENS, BCPHILIC, BCPHOBIC, DELP, DMS, DU001...
20                        [SLP, PS, H, T, U, V, QV, O3]
21                                     [AODANA, AODINC]
23    [CLOUD, DELP, EPV, H, O3, OMEGA, PHIS, PL, PS,...
24                                         [prec, rmse]
25    [time_bnds, Tair, Qair, PSurf, Wind_E, Wind_N,...
26    [aerosol_optical_thickness_550_count_land,

In [47]:
all_unique_vars = set().union(*subset_df.data_variables)

keywords = ['time', 'latitude', 'longitude', 'num_obs', 'foo', 'crs']
filtered_list = [item for item in all_unique_vars if not any(keyword in item.lower() for keyword in keywords)]
print(len(filtered_list))

509


In [36]:
with open('titiler_cmr_compatibility/known_vars.txt', 'w') as f:
    for item in filtered_list:
        f.write(f"{item}\n")

In [11]:
# !aws s3 cp s3://nasa-veda-scratch/aimee/tiling_results.parquet \
#     s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet

copy: s3://nasa-veda-scratch/aimee/tiling_results.parquet to s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet
